In [ ]:
# Default locations if parameter not passed
benchmark_data_path = "output.pkl"
output_folder = "./output"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from rich import print as rprint
import seaborn as sns
import os
from scipy.stats import bootstrap

benchmark_data = pickle.load(open(benchmark_data_path, "rb"))

df = pd.DataFrame(benchmark_data["dataframe"])

# create the output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


rprint(df.columns)
rprint(df.head())
rprint(df.tail())

In [ ]:
sns.set_theme()

colors = [
    "#7f7f7f",  # grey
    "#621dac",  # main purple
    "#c5702d",  # orange
    "#000000",  # black,
    "#099892",  # teal
    "#ffd400",  # yellow
    "#7e57c4",  # pink/purple,
]
colors_backup = colors.copy()

In [ ]:
def calculate_bootstrap_error(data):
    n_runs = benchmark_data["config"].metadata.runs
    if n_runs < 2:
        return 0
    res = bootstrap(
        (data,), np.mean, confidence_level=0.95, n_resamples=1000, method="basic"
    )
    return res.standard_error

In [ ]:
bench_names = df["name"].unique()
run_metadata = benchmark_data["config"].applications

for bench in bench_names:
    bench_metadata = run_metadata[bench]
    bench_plots = bench_metadata.plot
    n_runs = benchmark_data["config"].metadata.runs

    # Create one grouped figure per benchmark
    fig_grouped, ax_grouped = plt.subplots(figsize=(10, 6))

    for plot_setup in bench_plots:
        bench_df = df[df["name"] == bench].copy()
        print(bench)
        y_ax = plot_setup.y_axis
        x_ax = plot_setup.x_axis
        group_by = plot_setup.group_by
        title = plot_setup.title

        # Ensure Y axis is numeric and drop NaN columns / duplicates
        bench_df[y_ax] = pd.to_numeric(bench_df[y_ax], errors='coerce')
        bench_df = bench_df.dropna(axis=1)
        bench_df = bench_df.loc[:, ~bench_df.columns.duplicated()]

        # Group by and calculate mean and bootstrap error, then sort by x_ax
        grouped = (
            bench_df.groupby(group_by)[y_ax]
            .agg(["mean", calculate_bootstrap_error])
            .reset_index()
            .sort_values(by=x_ax)
        )

        rprint(grouped)

        # ----- Individual Figure for this plot_setup -----
        fig_ind, ax_ind = plt.subplots(figsize=(10, 6))
        ax_ind.errorbar(
            grouped[x_ax],
            grouped["mean"],
            yerr=grouped["calculate_bootstrap_error"],
            fmt="o-",
            capsize=5,
            label=f"{bench}-{y_ax}",
        )
        ax_ind.set_xlabel(x_ax)
        ax_ind.set_ylabel(y_ax)
        ax_ind.set_title(f"{title} | {n_runs} runs")
        ax_ind.grid(True)
        ax_ind.legend()
        ax_ind.set_xticks(grouped[x_ax])
        fig_ind.tight_layout()
        fig_ind.savefig(f"{output_folder}/{bench}_line_{y_ax}_{x_ax}_individual.pdf")
        plt.close(fig_ind)
        # -----------------------------------------------------

        # Add the same plot_setup to the grouped figure
        ax_grouped.errorbar(
            grouped[x_ax],
            grouped["mean"],
            yerr=grouped["calculate_bootstrap_error"],
            fmt="o-",
            capsize=5,
            label=f"{bench}-{y_ax}",
        )

    # Final adjustments and saving grouped plot
    ax_grouped.set_xlabel(x_ax)
    ax_grouped.set_ylabel(y_ax)
    ax_grouped.set_title(f"{title} | {n_runs} runs")
    ax_grouped.grid(True)
    ax_grouped.legend()
    ax_grouped.set_xticks(grouped[x_ax])
    fig_grouped.tight_layout()
    fig_grouped.savefig(f"{output_folder}/{bench}_line_{y_ax}_{x_ax}-GROUPED.pdf")
    plt.show()


In [ ]:
import numpy as np   # needed for x position computation

bench_names = df["name"].unique()
run_metadata = benchmark_data["config"].applications

for bench in bench_names:
    bench_metadata = run_metadata[bench]
    bench_plots = bench_metadata.plot

    # Create one grouped figure per bench to accumulate all plot_setups side by side
    fig_grouped = plt.figure(figsize=(10, 6))
    n_plots = len(bench_plots)

    for i, plot_setup in enumerate(bench_plots):
        bench_df = df[df["name"] == bench].copy()
        print(bench)
        y_ax = plot_setup.y_axis
        x_ax = plot_setup.x_axis
        group_by = plot_setup.group_by
        title = plot_setup.title
        n_runs = benchmark_data["config"].metadata.runs

        # Ensure the Y axis is numeric
        bench_df[y_ax] = pd.to_numeric(bench_df[y_ax])
        columns_with_nan = bench_df.columns[bench_df.isna().any()].tolist()

        # Drop columns with NaN and any duplicated columns
        bench_df = bench_df.drop(columns=columns_with_nan)
        bench_df = bench_df.loc[:, ~bench_df.columns.duplicated()]

        # Group by and calculate mean and bootstrap error
        grouped = (
            bench_df.groupby(group_by)[y_ax]
            .agg(["mean", calculate_bootstrap_error])
            .reset_index()
        )

        rprint(grouped)

        # Prepare x positions and offset for side-by-side bars (for grouped figure)
        x_positions = np.arange(len(grouped))
        bar_width = 0.8 / n_plots
        offset = (i - (n_plots - 1) / 2) * bar_width

        # --- Plot on the grouped figure ---
        # Activate the grouped figure before plotting
        plt.figure(fig_grouped.number)
        bars = plt.bar(
            x_positions + offset,
            grouped["mean"],
            width=bar_width,
            yerr=grouped["calculate_bootstrap_error"],
            capsize=5,
            label=f"{bench}-{y_ax}",
            edgecolor="black"
        )
        for bar in bars:
            bar.set_linewidth(1.5)
            bar.set_hatch("x")
        # Set basic labels and style (will be applied multiple times but is harmless)
        plt.xlabel(x_ax)
        plt.ylabel(y_ax)
        plt.grid(True)
        plt.legend()
        plt.title(f"{title} | {n_runs} runs")
        # ------------------------------------

        # --- Create individual figure for this plot_setup ---
        fig_ind = plt.figure(figsize=(10, 6))
        bars_ind = plt.bar(
            x_positions,
            grouped["mean"],
            width=0.8,
            yerr=grouped["calculate_bootstrap_error"],
            capsize=5,
            label=f"{bench}-{y_ax}",
            edgecolor="black"
        )
        for bar in bars_ind:
            bar.set_linewidth(1.5)
            bar.set_hatch("x")

        plt.xlabel(x_ax)
        plt.ylabel(y_ax)
        plt.grid(True)
        plt.legend()
        plt.title(f"{title} | {n_runs} runs")
        plt.xticks(x_positions, grouped[x_ax])
        plt.tight_layout()
        plt.savefig(f"{output_folder}/{bench}_bar_{y_ax}_{x_ax}_individual.pdf")
        plt.close(fig_ind)
        # -------------------------------------------------------

    # Final adjustments and saving for grouped figure
    plt.figure(fig_grouped.number)
    plt.xticks(x_positions, grouped[x_ax])
    plt.tight_layout()
    fig_grouped.savefig(f"{output_folder}/{bench}_bar_{y_ax}_{x_ax}-GROUPED.pdf")
    plt.show()
